In [ ]:
import pandas as pd 
import numpy as np
import glob  
import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

#### BASES CLARO COBRANÇA INTERSIC

caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES CLARO"
arquivos = glob.glob(caminho + "/*.txt")
#Lista de larguras do layout NET COB
lista_de_larguras = [50,50,20,50,20,50,50,100,20,50,9,9,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,17,17,19,18,8,11]
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_fwf(item, encoding='iso-8859-1', chunksize=1000000, sep=";",widths = lista_de_larguras, dtype='str')
        for df in chunks:
            dfList.append(df)

# Criando DataFrame com concat e a lista criada a cima
bases_claro = pd.concat(dfList, sort=False, ignore_index=False)

bases_claro = bases_claro.loc[:,['CPF','FILA']]

# TRATANDO CPF E CNPJ, INCLUINDO ZEROS A ESQUERDA.
df_cpfs_correto = bases_claro[bases_claro['CPF'].str.len()==11]
df_cnpjs_correto = bases_claro[bases_claro['CPF'].str.len()==14]
df_cpfs_corrigir = bases_claro[bases_claro['CPF'].str.len()<11]
df_cpfs_corrigir['CPF'] = df_cpfs_corrigir['CPF'].apply(lambda x: '{0:0>11}'.format(x))
df_cnpj_corrigir = bases_claro[bases_claro['CPF'].str.len()>11]
df_cnpj_corrigir['CPF']  = df_cnpj_corrigir['CPF'].apply(lambda x: '{0:0>14}'.format(x))
bases_claro = pd.concat([df_cpfs_correto,df_cpfs_correto,df_cpfs_corrigir,df_cnpj_corrigir],ignore_index=True)\
.drop_duplicates(['CPF','FILA'])
bases_claro

In [ ]:
########################################## LENDO A BASE OPERACIONAL PRONTA PARA TRABALHAR
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO"
base_operacional_hugo = pd.read_csv(caminho+"/BASE_OPERACIONAL_TELECOB_HUGO.txt", sep=';', dtype=str,decimal=',')

base_operacional_hugo['ATRASO'] = base_operacional_hugo['ATRASO'].astype('int16')
base_operacional_hugo['BASE'] = base_operacional_hugo['BASE'].apply(lambda x: x.replace('Claro MÃ³vel','Claro Movel'))
base_operacional_hugo = base_operacional_hugo.assign(FILA="")



###################### CRIANDO OS LAYOUTS PARA OS MAILINGS
preditivo_higienizado_claro = ["CPF","NOME_x","DEVEDOR","BASE","FILA_y","ATRASO","SALDO_ABERTO","TELEFONE_HIGIENIZADO"]
preditivo_validados_claro = ["CPF","NOME_x","DEVEDOR","BASE","FILA_y","ATRASO","SALDO_ABERTO",'TELEFONE',"TELEFONE_HIGIENIZADO"]
preditivo_telefones_intersic = ["CPF","NOME_x","DEVEDOR","BASE","FILA_y","ATRASO","SALDO_ABERTO",'TELEFONE']


In [ ]:
base_operacional_hugo = base_operacional_hugo.loc[:,preditivo_validados_claro]
base_operacional_hugo

In [ ]:
bases_claro_filtrada = pd.merge(base_operacional_hugo, bases_claro, right_on=['CPF'], left_on=['CPF'], how="inner")
bases_claro_filtrada


In [ ]:
mailing_hgienizado_claro = bases_claro_filtrada.loc[:,preditivo_higienizado_claro]
mailing_contratante_claro = bases_claro_filtrada.loc[:,preditivo_telefones_intersic]

In [ ]:
mailing_hgienizado_claro = mailing_hgienizado_claro.query("BASE != 'GEVENUE' and BASE != 'NET' ")

In [ ]:
# mailing_hgienizado_claro = mailing_hgienizado_claro.query("TELEFONE_HIGIENIZADO.notnull()").drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
# mailing_hgienizado_claro = mailing_hgienizado_claro.query("BASE != 'GEVENUE' and BASE != 'NET' ")

# mailing_contratante_claro = mailing_contratante_claro.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
# mailing_contratante_claro = mailing_contratante_claro.query("BASE != 'GEVENUE' and BASE != 'NET' ")

# display(mailing_hgienizado_claro)
# display(mailing_contratante_claro)
mailing_hgienizado_claro.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Maio\26\Jorgin da Clock/mailing_hgienizado_claro.csv",index=False,sep=';')
mailing_contratante_claro.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Maio\26\Jorgin da Clock/mailing_contratante_claro.csv",index=False,sep=';')

In [ ]:
mailing_hgienizado_claro

In [ ]:
import pandas as pd 
import numpy as np
import glob  
import warnings
#### BASES NET COBRANÇA INTERSIC

caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES NET"
arquivos = glob.glob(caminho + "/*.txt")
#Lista de larguras do layout NET COB
lista_de_larguras = [50,50,20,50,20,50,50,100,20,50,9,9,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,17,17,19,18,11,20]
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_fwf(item, encoding='iso-8859-1', chunksize=1000000, sep=";",widths = lista_de_larguras, dtype='str')
        for df in chunks:
            dfList.append(df)
            
###########################################################################################################################################    

# Criando DataFrame com concat e a lista criada a cima
bases_net = pd.concat(dfList, sort=False, ignore_index=False)

bases_net = bases_net.loc[:,['CPF','FILA']]

bases_net




In [ ]:
pd.merge(base_operacional_hugo, bases_net, right_on=['CPF'], left_on=['CPF'], how="inner")

In [ ]:
bases_net_filtrada = pd.merge(base_operacional_hugo, bases_net, right_on=['CPF'], left_on=['CPF'], how="inner")
bases_net_filtrada

In [ ]:
bases_net_filtrada

In [ ]:
mailing_hgienizado_net = bases_net_filtrada.loc[:,preditivo_higienizado_claro]
mailing_contratante_net = bases_net_filtrada.loc[:,preditivo_telefones_intersic]

In [ ]:
mailing_hgienizado_net = mailing_hgienizado_net.query("TELEFONE_HIGIENIZADO.notnull()").drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])\
.query('BASE =="NET"')
mailing_hgienizado_net

mailing_contratante_net = mailing_contratante_net.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])\
.query('BASE =="NET"')
mailing_contratante_net

In [ ]:
display(mailing_hgienizado_net)
mailing_hgienizado_net.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Maio\26\Jorgin da Clock/mailing_hgienizado_net.csv",sep=';',index=False)